In [1]:
import numba as np
import pandas as pd

In [2]:
train_orig =  pd.read_csv('../datasets/train.csv')
test_orig =  pd.read_csv('../datasets/test_unique.csv')

df1 = train_orig[['question1']].copy()
df2 = train_orig[['question2']].copy()
df1_test = test_orig[['question1']].copy()
df2_test = test_orig[['question2']].copy()

df2.rename(columns={'question2': 'question1'}, inplace=True)
df2_test.rename(columns={'question2': 'question1'}, inplace=True)

train_questions = df1.append(df2)
train_questions = train_questions.append(df1_test)
train_questions = train_questions.append(df2_test)
train_questions.drop_duplicates(subset=['question1'], inplace=True)

train_questions.reset_index(inplace=True, drop=True)
questions_dict = pd.Series(train_questions.index.values, index=train_questions.question1.values).to_dict()

In [3]:
train_cp = train_orig.copy()
test_cp = test_orig.copy()
train_cp.drop(['qid1', 'qid2'], axis=1, inplace=True)

test_cp['is_duplicate'] = -1
test_cp.rename(columns={'test_id': 'id'}, inplace=True)
comb = pd.concat([train_cp, test_cp])

comb['q1_hash'] = comb['question1'].map(questions_dict)
comb['q2_hash'] = comb['question2'].map(questions_dict)

q1_vc = comb.q1_hash.value_counts().to_dict()
q2_vc = comb.q2_hash.value_counts().to_dict()

comb['q1_freq'] = comb['q1_hash'].map(lambda x: q1_vc.get(x, 0) + q2_vc.get(x, 0))
comb['q2_freq'] = comb['q2_hash'].map(lambda x: q1_vc.get(x, 0) + q2_vc.get(x,0))

train_comb = comb[comb['is_duplicate'] >= 0][['id', 'q1_hash', 'q2_hash', 'q1_freq', 'q2_freq', 'is_duplicate']]
test_comb = comb[comb['is_duplicate'] < 0][['id', 'q1_hash', 'q2_hash', 'q1_freq', 'q2_freq']]

In [4]:
train_comb.corr()

,id,q1_hash,q2_hash,q1_freq,q2_freq,is_duplicate
id,1.000000,0.692730,0.286969,-0.001605,-0.000777,-0.008784
q1_hash,0.692730,1.000000,0.492993,-0.341774,-0.202545,-0.206498
q2_hash,0.286969,0.492993,1.000000,-0.392601,-0.466435,-0.349624
q1_freq,-0.001605,-0.341774,-0.392601,1.000000,0.494314,0.296619
q2_freq,-0.000777,-0.202545,-0.466435,0.494314,1.000000,0.198609
is_duplicate,-0.008784,-0.206498,-0.349624,0.296619,0.198609,1.000000


In [5]:
train_comb.head()

,id,q1_hash,q2_hash,q1_freq,q2_freq,is_duplicate
0,0,0,290457,1,2,0
1,1,1,16739,8,3,0
2,2,2,290458,2,1,0
3,3,3,290459,1,1,0
4,4,4,290460,3,1,0


In [6]:
test_comb.head()

,id,q1_hash,q2_hash,q1_freq,q2_freq
0,0,537361,2680914,1,1
1,1,537362,1258711,2,2
2,2,537363,2680915,1,1
3,3,537364,2680916,1,1
4,4,537365,2680917,1,1


In [7]:
columns = ['id', 'q1_hash', 'q2_hash', 'q1_freq', 'q2_freq']
train_comb[columns].to_csv("../datasets/train_magic_feature_v1.csv", index=False)
test_comb[columns].to_csv("../datasets/test_magic_feature_v1.csv", index=False)